In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI特征存储进行在线和批量预测（传统版）

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/sdk-feature-store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/sdk-feature-store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
    </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/feature_store/sdk-feature-store.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

这份笔记介绍了Vertex AI特征存储（传统版），这是一个托管的云服务，供机器学习工程师和数据科学家存储、提供、管理和分享大规模的机器学习特征。

这份笔记假设您了解基本的谷歌云概念，比如[项目](https://cloud.google.com/storage/docs/projects)，[存储](https://cloud.google.com/storage)和[Vertex AI](https://cloud.google.com/vertex-ai/docs)。一些机器学习知识也会有帮助，但不是必需的。

了解更多关于[Vertex AI特征存储](https://cloud.google.com/vertex-ai/docs/featurestore)。

### 目标

在这个笔记本中，您将学习如何使用`Vertex AI Feature Store (Legacy)`来导入特征数据，并访问这些特征数据用于在线服务和离线任务，比如训练。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Feature Store (Legacy)`

执行的步骤包括：

- 创建`Featurestore`、`EntityType`和`Feature`资源。
- 将特征数据导入到`Featurestore`资源中。
- 使用导入的特征进行在线预测请求。
- 在离线工作中访问导入的特征，比如训练作业。
- 使用流式导入来导入少量数据。

数据集

这个笔记本在所有会话中都使用一部电影推荐数据集作为示例。任务是训练一个模型，预测用户是否会观看一部电影，并将该模型提供在线服务。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage
* Cloud BigQuery

了解 [Vertex AI
 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
 根据您的预期使用量生成成本估算。

## 安装

安装下面所需的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform -q

只有合作：取消下面的单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的 Google Cloud 项目

**无论您使用哪种笔记本环境，都需要按照以下步骤进行设置。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。您第一次创建帐户时，您将获得 $300 的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。{TODO：更新您教程所需的 API。编辑 API 名称，并更新链接以附加 API ID，用逗号分隔每个。例如，container.googleapis.com，cloudbuild.googleapis.com}

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的“REGION”变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明。

1. Vertex AI 工作台
* 无需操作，因为您已经完成了身份验证。

2. 本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐号或其他
* 参见如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import Feature, Featurestore

### 初始化Python的Vertex AI SDK

为您的项目初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 术语和概念

### Vertex AI特征存储（遗留）数据模型

Vertex AI特征存储（遗留）使用以下三个重要的层次概念组织数据：
```
特征存储 -> 实体类型 -> 特征
```
* **特征存储**: 存储特征的位置
* **实体类型**: 在特征存储下，实体类型描述了一个需要建模的对象，可以是真实的也可以是虚拟的。
* **特征**: 在实体类型下，特征描述了实体类型的属性

电影预测示例让您创建一个名为`movie_prediction`的特征存储。该存储具有两个实体类型。`users`和`movies`。`users`实体类型具有`age`，`gender`和`liked_genres`特征。`movies`实体类型具有`titles`，`genres`和`average rating`特征。

创建特征存储并定义模式

In [ ]:
FEATURESTORE_ID = "movie_prediction_unique"
INPUT_CSV_FILE = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv"
ONLINE_STORE_FIXED_NODE_COUNT = 1

### 创建特征存储

创建特征存储的方法会返回一个[长时间运行的操作](https://google.aip.dev/151)（LRO）。LRO会启动一个异步任务。对于其他API方法，如更新或删除特征存储，也会返回LRO。运行此代码单元会创建一个特征存储并打印过程日志。

In [ ]:
fs = Featurestore.create(
    featurestore_id=FEATURESTORE_ID,
    online_store_fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT,
    project=PROJECT_ID,
    location=REGION,
    sync=True,
)

使用以下函数调用来检索特征商店并检查它是否已创建。

In [ ]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

### 创建实体类型

实体类型可以在`Featurestore`类中创建。在下面，创建`users`和`movies`实体类型。进程日志被打印出来。

In [ ]:
# Create the `users` entity type
users_entity_type = fs.create_entity_type(
    entity_type_id="users",
    description="Users entity",
)

In [ ]:
# Create the `movies` entity type
movies_entity_type = fs.create_entity_type(
    entity_type_id="movies",
    description="Movies entity",
)

要检索实体类型或检查是否已创建，请使用`Featurestore`对象上的`get_entity_type`或`list_entity_types`方法。

In [ ]:
users_entity_type = fs.get_entity_type(entity_type_id="users")
movies_entity_type = fs.get_entity_type(entity_type_id="movies")
print(users_entity_type)
print(movies_entity_type)

In [ ]:
fs.list_entity_types()

### 创建特性
您可以在每个实体类型内创建特性。使用`create_feature`方法将特性添加到`users`和`movies`实体类型中。

In [ ]:
# To create one feature at a time, use:
users_feature_age = users_entity_type.create_feature(
    feature_id="age",
    value_type="INT64",
    description="User age",
)

users_feature_gender = users_entity_type.create_feature(
    feature_id="gender",
    value_type="STRING",
    description="User gender",
)

users_feature_liked_genres = users_entity_type.create_feature(
    feature_id="liked_genres",
    value_type="STRING_ARRAY",
    description="An array of genres this user liked",
)

使用 [`list_features`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/entity_type.py#L349) 方法来列出给定实体类型的所有特征。

In [ ]:
users_entity_type.list_features()

In [ ]:
movies_feature_configs = {
    "title": {
        "value_type": "STRING",
        "description": "The title of the movie",
    },
    "genres": {
        "value_type": "STRING",
        "description": "The genre of the movie",
    },
    "average_rating": {
        "value_type": "DOUBLE",
        "description": "The average rating for the movie, range is [1.0-5.0]",
    },
}

In [ ]:
movie_features = movies_entity_type.batch_create_features(
    feature_configs=movies_feature_configs,
)

## 搜索已创建的特征

虽然 `list_features` 方法允许您查看同一实体类型的所有特征，
但是 `Feature` 类中的 [`search`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/feature.py#L352) 方法跨所有特征存储和实体类型进行搜索，返回一个特征列表。这样您就可以发现其他人创建的特征。

您可以根据特征属性进行查询，包括特征 ID、实体类型 ID 和特征描述。您还可以通过特定的特征存储、特征值类型和/或标签进行筛选以限制结果。以下是一些搜索示例。

**使用`search`方法的示例**

使用以下代码片段搜索特征存储中的所有特征：

In [ ]:
my_features = Feature.search(query="featurestore_id={}".format(FEATURESTORE_ID))
my_features

现在，将搜索范围缩小到类型为“DOUBLE”的特征。

In [ ]:
double_features = Feature.search(
    query="value_type=DOUBLE AND featurestore_id={}".format(FEATURESTORE_ID)
)
double_features[0].gca_resource

或者，将搜索结果限制为具有其ID和类型中特定关键字的功能。

In [ ]:
title_features = Feature.search(
    query="feature_id:title AND value_type=STRING AND featurestore_id={}".format(
        FEATURESTORE_ID
    )
)
title_features[0].gca_resource

导入特征值

在您可以将特征值用于在线或离线服务之前，您需要先导入特征值。在这一步中，您将学习如何从云存储中导入特征值。您也可以从BigQuery或pandas数据框导入特征值。

### 源数据格式和布局

支持BigQuery表/Avro/CSV作为输入数据类型。无论您使用哪种格式，每个导入的实体*必须*具有一个ID。每个实体可以*选择性*具有一个时间戳，指定生成特征值的时间。此笔记本使用Avro作为输入，位于此公共[bucket](https://console.cloud.google.com/storage/browser/cloud-samples-data-us-central1/vertex-ai/feature-store/datasets)。Avro模式如下：

**对于`users`实体**：
```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
```

**对于`movies`实体**：
```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### 导入`users`实体类型的特征值

在导入时，在您的请求中指定以下内容：

* 要导入的特征的ID
* 数据源URI
* 数据源格式：BigQuery表/Avro/CSV

In [ ]:
USERS_FEATURES_IDS = [feature.name for feature in users_entity_type.list_features()]
USERS_FEATURE_TIME = "update_time"
USERS_ENTITY_ID_FIELD = "user_id"
USERS_GCS_SOURCE_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
)
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(USERS_FEATURES_IDS)

In [ ]:
users_entity_type.ingest_from_gcs(
    feature_ids=USERS_FEATURES_IDS,
    feature_time=USERS_FEATURE_TIME,
    entity_id_field=USERS_ENTITY_ID_FIELD,
    gcs_source_uris=USERS_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    sync=False,
)

### 为 `movies` 实体类型导入特征值

同样，将 `movies` 实体类型的特征值导入特征存储。

In [ ]:
MOVIES_FEATURES_IDS = [feature.name for feature in movies_entity_type.list_features()]
MOVIES_FEATURE_TIME = "update_time"
MOVIES_ENTITY_ID_FIELD = "movie_id"
MOVIES_GCS_SOURCE_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
)
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(MOVIES_FEATURES_IDS)

In [ ]:
movies_entity_type.ingest_from_gcs(
    feature_ids=MOVIES_FEATURES_IDS,
    feature_time=MOVIES_FEATURE_TIME,
    entity_id_field=MOVIES_ENTITY_ID_FIELD,
    gcs_source_uris=MOVIES_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    sync=False,
)

获取来自你的模型的在线预测

在线服务可为少量实体提供特征值。它旨在为延迟敏感的服务提供支持，比如在线模型预测。例如，对于一个电影服务，您可能希望快速展示当前用户最有可能观看的电影。

使用Python SDK，读取一个实体的特征值非常容易。默认情况下，SDK返回每个特征的最新值，即具有最新时间戳的特征值。

要读取特征值，请指定实体类型ID和要读取的特征。默认情况下，选择实体类型的所有特征。输出响应将显示选定的实体类型ID以及选定的特征值作为Pandas数据帧。

In [ ]:
users_entity_type.read(entity_ids="bob")

In [ ]:
movies_entity_type.read(entity_ids="movie_01", feature_ids="title")

每次请求读取多个实体

要从多个实体中读取特征值，请指定不同的实体类型ID。默认情况下，选择实体类型的所有特征。请注意，由于此SDK的延迟敏感性，建议在使用时仅获取少量实体。

In [ ]:
users_entity_type.read(entity_ids=["bob", "alice"])

In [ ]:
movies_entity_type.read(
    entity_ids=["movie_02", "movie_03", "movie_04"], feature_ids=["title, genres"]
)

现在你已经学会了如何获取导入的特征值用于在线服务，下一步是学习如何将导入的特征值用于离线应用场景。

## 从您的模型获取批量预测

批量服务用于获取大量特征值以进行高吞吐量处理，通常用于训练模型或批量预测。在本节中，您将学习如何通过使用Vertex AI 特征存储（传统版）中的批处理服务功能准备训练示例。

### 使用案例

**任务** 是准备一个训练数据集来训练一个模型，该模型可以预测一个给定用户是否会观看电影。为了实现这一目标，您需要两组输入：

* 特征: 您已经在特征存储中导入了。
* 标签: 记录了用户X观看了电影Y的地面真相数据。

具体而言，地面真实观察数据在表1中描述，期望的训练数据集在表2中描述。表2中的每一行是根据表1中的实体ID和时间戳连接导入特征值而生成的结果。在这个示例中，从`用户`中选择 `age`, `gender` 和 `liked_genres` 特征以及从`电影`中选择 `titles`, `genres` 与 `average_rating` 特征来训练模型。请注意，这两个表中只显示了正例，也就是，您可以想象有一个标签列，其值都为`True`。

[`batch_serve_to_bq`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L770) 将表1作为输入，从特征存储中连接所有所需的特征值，并为训练返回表2。

<h4 align="center">表1. 地面真实数据</h4>

users | movies | timestamp
----- | -------- | --------------------
alice  | Cinema Paradiso     | 2019-11-01T00:00:00Z
bob  | The Shining     | 2019-11-15T18:09:43Z
...   | ...      | ...


<h4 align="center">表2. 通过批量提供生成的预期训练数据</h4>

timestamp            | 实体类型_用户 | 年龄 | 性别 | 喜欢的类型 | 实体类型_电影 | 标题 | 电影类型 | 平均评分
-------------------- | ----------------- | --------------- | ---------------- | -------------------- | - | -------- | --------- | -----
2019-11-01T00:00:00Z | bob              | 35        | 男                | [动作, 犯罪]      | 电影_02            | The Shining | 恐怖 | 4.8
2019-11-01T00:00:00Z | alice             | 55        | 女                | [戏剧, 喜剧]      | 电影_03          | Cinema Paradiso | 浪漫 | 4.5 |
... | ... | ... | ... | ... | ... | ... | ... | ...

为什么需要时间戳？

请注意，在表2中有一个“时间戳”列，用来表示观察到地面真相时的时间。这是为了避免数据不一致。

例如，表2的第二行表示用户“alice”在“2019-11-01T00:00:00Z”观看了电影“天堂电影院”。特征存储为所有时间戳保留特征值，但在批量服务期间仅获取给定时间戳的特征值。那天，Alice可能已经54岁了，但现在Alice可能已经56岁；特征存储返回`age=54`作为Alice的年龄，而不是`age=56`，因为这是观察时间的特征值。同样，其他特征也可能是随时间变化的，比如`liked_genres`。

### 为输出创建 BigQuery 数据集

您需要在 `us-central1` 中创建一个 BigQuery 数据集来承载输出数据。输入您想要创建的数据集的名称，并指定您想要存储稍后创建的输出的表的名称。这些将在下一部分中使用。

**确保表名尚不存在**。

In [ ]:
from datetime import datetime

from google.cloud import bigquery

In [ ]:
# Output dataset
DESTINATION_DATA_SET = "movie_predictions"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

# Output table. Make sure that the table does NOT already exist; the BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "training_data"  # @param {type:"string"}

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

In [ ]:
# Create dataset
client = bigquery.Client(project=PROJECT_ID)
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

批量读取特征数值

组装请求，指定以下信息：

* 标签数据存放在哪里，即表1。
* 要读取哪些特征，即表2中的列名。

输出存储在BigQuery表中。

In [ ]:
SERVING_FEATURE_IDS = {
    # to choose all the features use 'entity_type_id: ['*']'
    "users": ["age", "gender", "liked_genres"],
    "movies": ["title", "average_rating", "genres"],
}

In [ ]:
fs.batch_serve_to_bq(
    bq_destination_output_uri=DESTINATION_TABLE_URI,
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_uri=INPUT_CSV_FILE,
)

LRO完成后，您应该能够在[BigQuery控制台](https://console.cloud.google.com/bigquery)中看到结果，作为之前创建的BigQuery数据集下的新表。

实时导入让您可以实时更新特征值。批量导入适用于导入大量数据且延迟高的情况，而流式导入适用于导入少量数据且延迟低的情况。编写的数据可以通过批量导出和在线提供服务来读取。

In [ ]:
# Since streaming import is public preview, the feature is available in aiplatform_v1beta1.
from google.cloud.aiplatform_v1beta1 import (
    FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient)
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1beta1.types import types as types_pb2

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
# Create client connection
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
data_client = FeaturestoreOnlineServingServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

In [ ]:
# Call `write_feature_values` to import data to `users` entity type.
data_client.write_feature_values(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "users"
    ),
    payloads=[
        featurestore_online_service_pb2.WriteFeatureValuesPayload(
            entity_id="1305",
            feature_values={
                "age": featurestore_online_service_pb2.FeatureValue(int64_value=34),
                "gender": featurestore_online_service_pb2.FeatureValue(
                    string_value="female"
                ),
                "liked_genres": featurestore_online_service_pb2.FeatureValue(
                    string_array_value=types_pb2.StringArray(values=["drama", "action"])
                ),
            },
        ),
        featurestore_online_service_pb2.WriteFeatureValuesPayload(
            entity_id="1306",
            feature_values={
                "age": featurestore_online_service_pb2.FeatureValue(int64_value=50),
                "gender": featurestore_online_service_pb2.FeatureValue(
                    string_value="male"
                ),
                "liked_genres": featurestore_online_service_pb2.FeatureValue(
                    string_array_value=types_pb2.StringArray(
                        values=["suspense", "comedy"]
                    )
                ),
            },
        ),
    ],
)

成功完成后，`write_feature_values` API 返回一个空响应。
同样，将数据导入到 `movies` 实体类型。

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

您也可以保留该项目，但通过运行以下代码删除特征存储和BigQuery数据集：

In [ ]:
# Delete Featurestore
fs.delete(force=True)

In [ ]:
# Delete BigQuery dataset
client = bigquery.Client(project=PROJECT_ID)
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))